In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

Funciona!


In [53]:
ingresos = pd.read_csv('../../sancor_salud/bajas/data/raw/historico_ingresos_201807301544.csv.gz')


,ID,P_KEY,FECHA_DESDE,CUENTA,SUB_CUENTA,EMPLEADOR,REMUNERACION,P_KEY.1,FIRST_ID
0,10,33613032059,20150917204854,336,13,32059,30217.02,33613032059,10
1,11,33613032059,20151019175221,336,13,32059,34682.35,33613032059,10
2,12,33613032059,20151119072643,336,13,32059,35093.02,33613032059,10


In [54]:
ingresos.columns = ingresos.columns.str.lower()

In [55]:
def ss_parse_datetime(data, format_):
    if not isinstance(data, pd.Series):
        raise ValueError('Data must be a Pandas Series')

    if data.dtype.kind not in ('f', 'i'):
        raise ValueError('Data must be off type float or int.')

    name = data.name
    ix = data.notnull() & (data != 0)
    new_data = pd.Series(index=data.index, dtype='datetime64[ns]')
    new_data[ix] = pd.to_datetime(data[ix].astype('int'), format=format_, errors='coerce')

    coerced_ix = (new_data.isnull() & ix)
    coerced_q = (coerced_ix).sum()
    return new_data


def ss_date(data):
    return ss_parse_datetime(data, '%Y%m%d')


def ss_datetime(data):
    return ss_parse_datetime(data, '%Y%m%d%H%M%S')


In [56]:
ingresos.fecha_desde = ss_datetime(ingresos.fecha_desde)

In [57]:
ingresos = ingresos[['fecha_desde', 'cuenta', 'empleador', 'remuneracion']].copy()

In [58]:
ingresos = ingresos[ingresos.fecha_desde.notnull()].copy()

In [76]:
def trasnform_code(code):
    bytes_ = str(code).encode('ascii')
    new_code = hash(bytes_) % 100_000_000_000
    return new_code

In [77]:
ingresos['empleador_en'] = ingresos.empleador.apply(trasnform_code)
ingresos.empleador.nunique(), ingresos.empleador_en.nunique()

(62201, 62201)

In [78]:
ingresos['cuenta_en'] = ingresos.cuenta.apply(trasnform_code)
ingresos.cuenta.nunique(), ingresos.cuenta_en.nunique()

(226992, 226992)

In [80]:
ingresos = ingresos.drop(columns=['empleador', 'cuenta'])

In [81]:
ingresos = ingresos.rename(columns={'empleador_en': 'empleador', 'cuenta_en': 'cuenta'})

In [87]:
ingresos = ingresos.reset_index(drop=True)

In [89]:
to_null = np.random.randint(0, len(ingresos), size=len(ingresos) // 20)
ingresos.loc[to_null, 'remuneracion'] = np.nan

In [162]:
ingresos.to_csv('ingresos.csv.gz', compression='gzip', index=False)

In [94]:
ingresos.remuneracion.describe()

count    3.572367e+06
mean     2.058011e+04
std      3.073897e+04
min      0.000000e+00
25%      1.299304e+04
50%      1.829055e+04
75%      2.486579e+04
max      9.710000e+06
Name: remuneracion, dtype: float64

In [184]:
stock = pd.read_csv('../../sancor_salud/bajas/data/raw/stock_subcuentas_201807301607.csv.gz')

In [185]:
stock.columns = stock.columns.str.lower()

In [186]:
stock = stock[['cuenta', 'fecha_baja_sistema', 'motivo_baja']].drop_duplicates()

In [187]:
stock = stock[stock.fecha_baja_sistema > 0].copy()

In [188]:
stock['fecha_baja'] = ss_date(stock.fecha_baja_sistema)

In [189]:
stock['cuenta_en'] = stock.cuenta.apply(trasnform_code)
stock.cuenta.nunique(), stock.cuenta_en.nunique()

(115053, 115053)

In [190]:
import this

In [191]:
import codecs
s = codecs.decode(this.s, 'rot-13')


In [192]:
lines = [x for x in s.split('\n') if len(x) > 0]

In [193]:
import random

In [194]:
remaining_lines = lines[:]
for m in stock.motivo_baja.unique():
    if len(remaining_lines) == 0:
        remaining_lines = lines[:]
    stock.loc[stock.motivo_baja == m, 'motivo_en'] = random.choice(remaining_lines)

In [199]:
stock = stock[stock.motivo_baja!='RESCISION DE CONTRATO/CONVENIO']

In [200]:
stock = stock[['cuenta_en', 'motivo_en', 'fecha_baja']].copy()

In [201]:
stock.columns = ['cuenta', 'motivo_baja', 'fecha_baja']

In [202]:
stock.head()

,cuenta,motivo_baja,fecha_baja
2,43653210508,Explicit is better than implicit.,2018-06-27
18,53077128540,Unless explicitly silenced.,2017-04-25
31,36001999808,Unless explicitly silenced.,2017-01-09
33,12420534830,Readability counts.,2016-12-12
39,55097901246,Flat is better than nested.,2017-06-21


In [205]:
stock = stock.sample(frac=0.5)

In [207]:
stock.to_csv('bajas.csv.gz', compression='gzip', index=False)